In [ ]:
# import libraries
import pandas as pd
!pip install tensorflow-datasets
!pip install tensorflow
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping

print(tf.__version__)

2.18.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-09-04 18:00:38--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  1013KB/s    in 0.3s    

2025-09-04 18:00:39 (1013 KB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-09-04 18:00:39--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
df_train = pd.read_csv(train_file_path, sep= "\t" ,header=None, names=["class" , "message"])
print(df_train.head())


  class                                            message
0   ham  ahhhh...just woken up!had a bad dream about u ...
1   ham                           you can never do nothing
2   ham  now u sound like manky scouse boy steve,like! ...
3   ham  mum say we wan to go then go... then she can s...
4   ham  never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
df_test = pd.read_csv(test_file_path, sep= "\t",header=None, names=["class" , "message"])
print(df_test.head())

  class                                            message
0   ham  i am in hospital da. . i will return home in e...
1   ham         not much, just some textin'. how bout you?
2   ham  i probably won't eat at all today. i think i'm...
3   ham  don‘t give a flying monkeys wot they think and...
4   ham                                who are you seeing?


In [ ]:
print(len(df_train))
print(len(df_test))

4179
1392


In [ ]:
df_train['class'] = df_train['class'].replace({'ham':0 ,'spam':1})
df_test['class'] = df_test['class'].replace({'ham':0 ,'spam':1})

/tmp/ipython-input-6-2832180405.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['class'] = df_train['class'].replace({'ham':0 ,'spam':1})
/tmp/ipython-input-6-2832180405.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['class'] = df_test['class'].replace({'ham':0 ,'spam':1})


In [ ]:
'''
vocabulary_dict = {}
for messgae in df_train['message']:
  for vocabulary in messgae.split():
    if vocabulary not in vocabulary_dict:
      vocabulary_dict[vocabulary] = 1
    else:
      vocabulary_dict[vocabulary] += 1
'''

"\nvocabulary_dict = {}\nfor messgae in df_train['message']:\n  for vocabulary in messgae.split():\n    if vocabulary not in vocabulary_dict:\n      vocabulary_dict[vocabulary] = 1\n    else:\n      vocabulary_dict[vocabulary] += 1\n"

In [ ]:
VOCAB_SIZE = 10000
MAX_LENGTH = 100

In [ ]:
#encoded_train_message = [one_hot(d, VOCAB_SIZE) for d in df_train['message']]
#padded_train_message = pad_sequences(encoded_train_message, maxlen=MAX_LENGTH, padding='post')
#encoded_test_message = [one_hot(d, VOCAB_SIZE) for d in df_test['message']]
#padded_test_message = pad_sequences(encoded_test_message, maxlen=MAX_LENGTH, padding='post')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(df_train['message'])

padded_train_message = pad_sequences(tokenizer.texts_to_sequences(df_train['message']),
                                     maxlen=MAX_LENGTH, padding='post')
padded_test_message = pad_sequences(tokenizer.texts_to_sequences(df_test['message']),
                                    maxlen=MAX_LENGTH, padding='post')


In [ ]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, 100, input_length=MAX_LENGTH))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=25,
                        verbose=1, mode='max', restore_best_weights=True)
model.fit(padded_train_message, df_train['class'],
          validation_data=(padded_test_message, df_test['class']),
          callbacks=[monitor], epochs=50, verbose=2)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50
131/131 - 4s - 33ms/step - accuracy: 0.8605 - loss: 0.3711 - val_accuracy: 0.8657 - val_loss: 0.3586
Epoch 2/50
131/131 - 4s - 34ms/step - accuracy: 0.8660 - loss: 0.3414 - val_accuracy: 0.8657 - val_loss: 0.3290
Epoch 3/50
131/131 - 4s - 28ms/step - accuracy: 0.8660 - loss: 0.3075 - val_accuracy: 0.8657 - val_loss: 0.2875
Epoch 4/50
131/131 - 1s - 10ms/step - accuracy: 0.8703 - loss: 0.2536 - val_accuracy: 0.8858 - val_loss: 0.2248
Epoch 5/50
131/131 - 1s - 11ms/step - accuracy: 0.9218 - loss: 0.1883 - val_accuracy: 0.9296 - val_loss: 0.1717
Epoch 6/50
131/131 - 2s - 18ms/step - accuracy: 0.9603 - loss: 0.1364 - val_accuracy: 0.9698 - val_loss: 0.1342
Epoch 7/50
131/131 - 2s - 19ms/step - accuracy: 0.9718 - loss: 0.1044 - val_accuracy: 0.9684 - val_loss: 0.1076
Epoch 8/50
131/131 - 2s - 18ms/step - accuracy: 0.9761 - loss: 0.0853 - val_accuracy: 0.9749 - val_loss: 0.0934
Epoch 9/50
131/131 - 1s - 10ms/step - accuracy: 0.9818 - loss: 0.0719 - val_accuracy: 0.9705 - val_loss:

In [ ]:
def predict_message(pred_text):
    class_dict = {0: "ham", 1: "spam"}

    # Use the trained tokenizer
    encoded_message = tokenizer.texts_to_sequences([pred_text])
    padded_message = pad_sequences(encoded_message, maxlen=MAX_LENGTH, padding='post')

    pred = model.predict(padded_message, verbose=0)[0][0]  # single float
    label = class_dict[int(np.round(pred))]

    return [pred, label]

# Example
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

[np.float32(0.020044276), 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
